In [ ]:
# render the svg file
import sqlite3
import os
import svgwrite
import cv2

def render_svg0(uid, image, cursor):
    filename = "./svg_var_width/" + uid + "&" + image + ".svg"
    img = cv2.imread('./image/' + image + '.jpg')
    img_size = (img.shape[1], img.shape[0])
    
    dwg = svgwrite.Drawing(filename, size=img_size)
    cursor.execute("select * from StrokeData where image='%s' and username='%s'" % (image, uid))
    strokedata = cursor.fetchall()
    for i in range(len(strokedata)):
#         print(strokedata[i])
        txy = strokedata[i][5].split(",")
        color = "#000000"
#         if int(image) > 63: 
#             width = 15
#         else:
#             width = 5
        width = max(img.shape[1], img.shape[0])//200
#         print(img_size, width)
        d = "M"
        for j in range(0, len(txy), 3):
            if j >= 3:
                d = d + "L"
            d = d + txy[j+1] + "," + txy[j+2]
        w = dwg.path(d=d, fill="none", stroke=color, style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0); stroke-linecap: round; stroke-linejoin: round; stroke-width: %s;" % (width))
        dwg.add(w)
    dwg.save()

try:
    conn = sqlite3.connect("dura-db.db")
except Error as e:
    print(e)
    
cursor = conn.cursor()

cursor.execute("select * from UserData")
userdata = cursor.fetchall()
for ud in userdata:
    uid = ud[1]
    images = ud[4]
    if images is not None: 
        images = images.split(",")
        for image in images: 
            render_svg0(uid, image, cursor)
    print(uid + " converted")

In [ ]:
# turning the svg file to png
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF, renderPM

# svgpath = r'sketches/'
svgpath = r'svg_var_width/'
pngpath = r'png_var_width/'
svgdir = os.path.basename(svgpath)

for filename in os.listdir(svgpath):
    name, ext = filename.split('.')
    drawing = svg2rlg(svgpath+filename)
    pngname = pngpath+name+'.png'
    renderPM.drawToFile(drawing, pngname, fmt="PNG")